In [25]:
import pandas as pd

# Data Import

FEATURES = ['capacity_bytes', 'smart_1_normalized', 'smart_2_normalized', 'smart_3_normalized', 'smart_4_normalized', 'smart_5_normalized', 'smart_7_normalized', 'smart_8_normalized', 'smart_9_normalized', 'smart_10_normalized', 'smart_11_normalized', 'smart_12_normalized', 'smart_13_normalized', 'smart_15_normalized', 'smart_16_normalized', 'smart_17_normalized', 'smart_22_normalized', 'smart_23_normalized', 'smart_24_normalized', 'smart_168_normalized', 'smart_170_normalized', 'smart_173_normalized', 'smart_174_normalized', 'smart_177_normalized', 'smart_179_normalized', 'smart_181_normalized', 'smart_182_normalized', 'smart_183_normalized', 'smart_184_normalized', 'smart_187_normalized', 'smart_188_normalized', 'smart_189_normalized', 'smart_190_normalized', 'smart_191_normalized', 'smart_192_normalized', 'smart_193_normalized', 'smart_194_normalized', 'smart_195_normalized', 'smart_196_normalized', 'smart_197_normalized', 'smart_198_normalized', 'smart_199_normalized', 'smart_200_normalized', 'smart_201_normalized', 'smart_218_normalized', 'smart_220_normalized', 'smart_222_normalized', 'smart_223_normalized', 'smart_224_normalized', 'smart_225_normalized', 'smart_226_normalized', 'smart_231_normalized', 'smart_232_normalized', 'smart_233_normalized', 'smart_235_normalized', 'smart_240_normalized', 'smart_241_normalized', 'smart_242_normalized', 'smart_250_normalized', 'smart_251_normalized', 'smart_252_normalized', 'smart_254_normalized', 'smart_255_normalized', 'failure']
FEATURES_BACKBLAZE = ['smart_5_normalized', 'smart_187_normalized', 'smart_188_normalized', 'smart_197_normalized', 'smart_198_normalized', 'failure']

data = pd.read_csv("./data/2019-07-01.csv", usecols=FEATURES_BACKBLAZE)

In [26]:
from sklearn.model_selection import train_test_split
import torch 

# Data Preparation
X = data.drop('failure', axis=1).values
y = data['failure'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

In [27]:
from torch import nn
from torch import optim

# Model Definition

input_size = len(FEATURES_BACKBLAZE) - 1
hidden_sizes = [int(input_size*4), int(input_size*2)]
output_size = 1

model = nn.Sequential(nn.Linear(input_size, hidden_sizes[0]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[0], hidden_sizes[1]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[1], output_size),
                      nn.Softmax(dim=1))

loss = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [28]:
# Model Training

epochs = 10

for i in range(epochs):
   y_hat = model.forward(X_train)
   output = loss(y_hat, y_train)

   if i % 2 == 0:
       print(f'Epoch: {i} Loss: {output}')

   optimizer.zero_grad()
   output.backward()
   optimizer.step()

RuntimeError: RuntimeError: [enforce fail at CPUAllocator.cpp:64] . DefaultCPUAllocator: can't allocate memory: you tried to allocate 31336080400 bytes. Error code 12 (Cannot allocate memory)


In [29]:
import torch
# Model Evaluation

preds = []

with torch.no_grad():
   for val in X_test:
       y_hat = model.forward(val)
       preds.append(y_hat.argmax().item())

df = pd.DataFrame({'Y': y_test, 'YHat': preds})
df['Correct'] = [1 if corr == pred else 0 for corr, pred in zip(df['Y'], df['YHat'])]
df['Correct'].sum() / len(df)


IndexError: IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)